![Alt text](new.png)

# Calculate reference stress for each temperature at each strain rate

For each condition, calculate take the stress where strain is smaller than 0.08 and divide it by the reference stress at 298.15K

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score
from itertools import product  # Import product for grid search
import warnings
from matplotlib.backends.backend_pdf import PdfPages
warnings.simplefilter('ignore')

# Strain-rate dependent function fit with the new equation

In [ ]:
# Define the Polynomial-Log Hybrid Model
def strain_rate_dependent_stress(sr, C1, C2, C3, C4):
    return C1 * np.log(sr) + C2 * sr**C3 + C4 * (np.log(sr))**2

def fit_strain_rate_dependency(temperature_file, strain_rates, strain_rate_columns):
    # Load the temperature-specific file
    data = pd.read_csv(f"../{temperature_file}.csv")
    
    # Define a scaling factor for better readability in the plot legend
    scale_factor = 1

    # List to store the results
    results = []

    # PDF file to save plots
    pdf_filename = f"Plots_{temperature_file}.pdf"
    
    with PdfPages(pdf_filename) as pdf:
        # Loop through all rows in the dataset for strain-rate dependency fitting
        for i in range(len(data)):
            # Extract stress data for other strain rates
            stress_values = data.iloc[i][strain_rate_columns]
            
            # Initial guess and bounds for C1, C2, C3, and C4
            initial_guess_sr = [0.1, 0.1, 0.1, 0.1] 
            
            # Perform the curve fitting
            try:
                params_sr, _ = curve_fit(
                    lambda sr, C1, C2, C3, C4: strain_rate_dependent_stress(sr, C1, C2, C3, C4),
                    strain_rates, stress_values, p0=initial_guess_sr, maxfev=10000000, method='dogbox'
                )
            except RuntimeError:
                print(f"Could not fit strain-rate dependency for row {i}")
                continue
            
            # Calculate residuals and R-squared
            stress_fit = strain_rate_dependent_stress(strain_rates, *params_sr)
            residuals = stress_values - stress_fit
            ss_res = np.sum(residuals**2)
            ss_tot = np.sum((stress_values - np.mean(stress_values))**2)
            r_squared = 1 - (ss_res / ss_tot)
            
            # Store results for exporting
            results.append({
                "PEEQ": data['strain'].iloc[i],
                "C1": params_sr[0],
                "C2": params_sr[1],
                "C3": params_sr[2],
                "C4": params_sr[3],
                "R_squared": r_squared
            })
            
            # Plot only for specified rows with scaled C1, C2, C3, and C4 in legend
            plt_strain_rates = np.linspace(min(strain_rates), max(strain_rates), 1000)
            plt_stress_fit = strain_rate_dependent_stress(plt_strain_rates, *params_sr)
            plt.plot(strain_rates, stress_values, 'o', label='Data')
            plt.plot(plt_strain_rates, plt_stress_fit, '-', label='Fit')  # Fit plot
            plt.xlabel('Strain Rate')
            plt.ylabel('Stress')
            plt.title(f"PEEQ={data['strain'].iloc[i]} (R² = {r_squared:.3f})", fontweight='bold')
            #plt.legend([f'Data', f'Fit: C1={params_sr[0]:.2f}, C2={params_sr[1]:.2f}, C3={params_sr[2]:.2f}, C4={params_sr[3]:.2f}'], loc="lower right")
            plt.legend(['Data', 'Fit'], loc="upper right")
            pdf.savefig()  # Save the current figure to the PDF
            plt.close()

    # Export results to CSV
    results_df = pd.DataFrame(results)
    results_df.to_csv(f"Parameters_{temperature_file}_new.csv", index=False)


In [ ]:
temperature_file = 'Tmp298.15_RD'
strain_rates = [0.0001, 0.001, 0.01, 0.1, 1]
strain_rate_columns = ['ref_StrRt0.0001', 'ref_StrRt0.001', 'ref_StrRt0.01', 'ref_StrRt0.1', 'ref_StrRt1']
fit_strain_rate_dependency(temperature_file, strain_rates, strain_rate_columns)

temps = ['373.15', '473.15', '573.15']
strain_rates = [0.0001, 0.001, 0.01, 0.1]
strain_rate_columns = ['ref_StrRt0.0001', 'ref_StrRt0.001', 'ref_StrRt0.01', 'ref_StrRt0.1']
for temp in temps:
    fit_strain_rate_dependency(f"Tmp{temp}_RD", strain_rates, strain_rate_columns)

temperature_file = 'Tmp673.15_RD'
strain_rates = [0.0001, 0.001, 0.01, 0.1]
strain_rate_columns = ['ref_StrRt0.0001', 'ref_StrRt0.001', 'ref_StrRt0.01', 'ref_StrRt0.1']
fit_strain_rate_dependency(temperature_file, strain_rates, strain_rate_columns)

## Check r_squared values for accuracy

In [ ]:
def check_r_squared(temperature_files):
    """
    Checks R_squared values in the Parameters file for each temperature and reports the count
    and percentage of rows with R_squared below a specified threshold.
    
    Parameters:
    - temperature_files (list of str): List of temperature file names (without ".csv" extension).
    
    Output:
    - Prints the count and percentage of rows with R_squared < threshold for each temperature file.
    """
    threshold = 0.99 # adjust as needed

    for temp_file in temperature_files:
        # Load the data from the CSV file
        df = pd.read_csv(f"Parameters_{temp_file}_new.csv")
        
        # Count rows where R_squared is less than the threshold
        total_rows = len(df)
        count_high_r_squared = df[df['R_squared'] > threshold].shape[0]
        
        # Calculate the percentage of rows with low R_squared
        percentage_low_r_squared = (count_high_r_squared / total_rows) * 100 if total_rows > 0 else 0
        
        # Print results
        print(f"\nTemperature File: Parameters_{temp_file}.csv")
        if count_high_r_squared > 0:
            print(f"Number of rows with R_squared > {threshold}: {count_high_r_squared}")
            print(f"Percentage of rows with R_squared > {threshold}: {percentage_low_r_squared:.2f}%")
        else:
            print(f"All R_squared values are <= {threshold}.")

        r_min = min(df['R_squared'])
        r_max = max(df['R_squared'])
        r_a = sum(df['R_squared'])/total_rows
        print(f'Min = {r_min}. Max = {r_max}. Average = {r_a}')

# Usage
temperature_files = ['Tmp298.15_RD', 'Tmp373.15_RD', 'Tmp473.15_RD', 'Tmp573.15_RD', 'Tmp673.15_RD']  # List of temperature files to check
check_r_squared(temperature_files)

# Calibrate C1-C4 based on strain dependent

## Study relationship between C1-C4 vs strain

In [ ]:
# List of temperature files to load
temperature_files = ['Tmp298.15_RD', 'Tmp373.15_RD', 'Tmp473.15_RD', 'Tmp573.15_RD']

# Function to plot parameters based on condition
def plot_parameters(df, x_col, y_col, filter_condition, title, xlabel, ylabel):
    plt.figure(figsize=(7, 4))
    for temp_file in temperature_files:
        df = pd.read_csv(f"Parameters_{temp_file}_new.csv")
        if filter_condition == "x < 0.5":
            filtered_df = df[df[x_col] < 0.5]
        else:
            filtered_df = df
        plt.plot(filtered_df[x_col], filtered_df[y_col], label=f'{y_col} - {temp_file}')
    plt.xlabel(xlabel, fontsize=12)
    plt.ylabel(ylabel, fontsize=12)
    plt.title(f'{title}', fontweight='bold', fontsize=12)
    plt.legend(fontsize=10)
    plt.show()

#Usage
plot_parameters(df=None, x_col='PEEQ', y_col='C1', filter_condition="strain < 0.5",
                title='C1 vs Strain for All Temperatures', xlabel='Strain (PEEQ)', ylabel='C1')
plot_parameters(df=None, x_col='PEEQ', y_col='C2', filter_condition="strain < 0.5",
                title='C2 vs Strain for All Temperatures', xlabel='Strain (PEEQ)', ylabel='C2')
plot_parameters(df=None, x_col='PEEQ', y_col='C3', filter_condition="strain < 0.5",
                title='C3 vs Strain for All Temperatures', xlabel='Strain (PEEQ)', ylabel='C3')
plot_parameters(df=None, x_col='PEEQ', y_col='C4', filter_condition="strain < 0.5",
                title='C4 vs Strain for All Temperatures', xlabel='Strain (PEEQ)', ylabel='C4')

## Plot parameters against strain for video

In [ ]:
# List of temperature files to load
temperature_files = ['Tmp298.15_RD', 'Tmp373.15_RD', 'Tmp473.15_RD', 'Tmp573.15_RD']
# Parameters to plot
parameters = ['C1', 'C2', 'C3', 'C4']

# Load and combine data from all temperature files
all_data = []
for temp_file in temperature_files:
    df = pd.read_csv(f"Parameters_{temp_file}_new.csv")
    df['Temperature'] = temp_file  # Add temperature file info
    all_data.append(df)
combined_data = pd.concat(all_data)

# Generate PDFs for each parameter
for param in parameters:
    output_pdf = f"incremental_plots_{param}.pdf"
    with PdfPages(output_pdf) as pdf:
        # Get unique PEEQ values and sort them
        unique_peeq = sorted(combined_data['PEEQ'].unique())
        for i in range(1, len(unique_peeq) + 1):
            max_peeq = unique_peeq[i - 1]
            plt.figure(figsize=(15, 7))
            for temp_file in temperature_files:
                # Filter data for current temperature and up to the current PEEQ limit
                temp_data = combined_data[(combined_data['Temperature'] == temp_file) & (combined_data['PEEQ'] <= max_peeq)]
                plt.plot(temp_data['PEEQ'], temp_data[param], label=f'{param} - {temp_file}')
            plt.xlabel('Strain (PEEQ)', fontsize=12)
            plt.ylabel(param, fontsize=12)
            plt.title(f'{param} vs Strain (PEEQ ≤ {max_peeq:.3f})', fontweight='bold', fontsize=12)
            plt.legend(loc='lower right', fontsize=12)
            plt.xlim(left=0)
            pdf.savefig()
            plt.close()
    print(f"Incremental plots for {param} saved to {output_pdf}")


## Calibrate with dynamic threshold

In [ ]:
def calibrate_strain_dependent_dynamic(temp_file):
    data = pd.read_csv(f"Parameters_{temp_file}_new.csv")  # Replace with your actual file path
    strain = data['PEEQ']  # Strain values
    C1_values = data['C1']  # C1 values
    C2_values = data['C2']  # C2 values
    C3_values = data['C3']  # C1 values
    C4_values = data['C4']  # C2 values

    # Define the stretched exponential model
    def C1_func(x, a1, b1, c1, d1, e1, a2, b2, c2, a3, b3, c3,d3, e3,f3,g3, threshold1, threshold2):
        # Low strain: Oscillatory component
        small_strain = a1 * np.exp(b1 * x) + c1 + d1*x**2 + e1*x 
        
        # Middle strain: Oscillatory component for waves
        middle_strain = a3 * np.sin(b3 * x + c3) * np.exp(-d3 * x) + e3 * x**2 + f3 * x + g3 
        
        # High strain: Stable behavior
        high_strain = a2 * np.exp(b2 * x) + c2
        
        # Piecewise combination based on thresholds
        return np.where(
            x <= threshold1, small_strain,
            np.where(x <= threshold2, middle_strain, high_strain)
        )

    def C2_func(x, a, b, c, d, e, f, g):
        epsilon = 1e-6
        x_shifted = x + epsilon
        return (
            a * np.log(x_shifted) + 
            b * x_shifted**c + 
            d * np.exp(-e * x_shifted) + 
            f * np.exp(-((x - g)**2) / (2 * e**2))
        )


    def C3_func(x, a1, b1, c1, d1, a2, b2, c2, d2, a3,b3,c3, threshold1, threshold2):
        small_strain = a1*x**3 + b1*x**2 + c1*x + d1
        middle_strain = a2*x**3 + b2*x**2 + c2*x + d2
        high_strain = a3*x**2 + b3*x + c3
        # Piecewise combination based on thresholds
        return np.where(
            x <= threshold1, small_strain,
            np.where(x <= threshold2, middle_strain, high_strain)
        )


    def C4_func(x, a1, b1, c1, d1, a2, b2, c2, a3, b3, c3,d3, e3,f3,g3, threshold1, threshold2):
        # Low strain: Oscillatory component
        small_strain = a1 * np.exp(b1 * x) * np.cos(c1 * x) + d1
        
        # Middle strain: Oscillatory component for waves
        middle_strain = a3 * np.sin(b3 * x + c3) * np.exp(-d3 * x) + e3 * x**2 + f3 * x + g3 
        
        # High strain: Stable behavior
        high_strain = a2 * np.exp(b2 * x) + c2
        
        # Piecewise combination based on thresholds
        return np.where(
            x <= threshold1, small_strain,
            np.where(x <= threshold2, middle_strain, high_strain)
        )

    #if temp_file == 'Tmp573.15_RD':
        #initial_guess_C1 = [0.1, -100, 100, 0.1, 1, 0.01, -0.5, -0.2, 0.02, 10, 0, 10, 0.01, -0.1, 0, 0.003, 0.53]
    #else: 
    initial_guess_C1 = [0.1, -100, 100, 0.1, 1, 0.01, -0.5, -0.2, 0.02, 10, 0, 10, 0.01, -0.1, 0, 0.002, 0.04]
    initial_guess_C2 = [1, 10, 0.5, 1, 0.1, 1, 0.01]
    initial_guess_C3 = [1,1,1,1,1,1,1,1,1,1,1, 0.002, 0.04]
    initial_guess_C4 = [-0.1, -1000, 1000, 0, -0.01, -50, 0.01, 0.1, 1.0, 0.0, 0.1, 0.0, 1.0, 0.0, 0.002, 0.04]
    # Fit the stretched exponential model to C1
    popt_C1, _ = curve_fit(C1_func, strain, C1_values, p0=initial_guess_C1, maxfev=10000000)
    popt_C2, _ = curve_fit(C2_func, strain, C2_values, p0=initial_guess_C2, maxfev=10000000)
    popt_C3, _ = curve_fit(C3_func, strain, C3_values, p0=initial_guess_C3,maxfev=10000000)
    popt_C4, _ = curve_fit(C4_func, strain, C4_values, p0=initial_guess_C4, maxfev=10000000)

    # Extract optimized thresholds
    threshold1_C1, threshold2_C1 = popt_C1[-2], popt_C1[-1]
    threshold1_C3, threshold2_C3 = popt_C3[-2], popt_C3[-1]
    threshold1_C4, threshold2_C4 = popt_C4[-2], popt_C4[-1]

    print(f"Optimized Thresholds for C1: Threshold1={threshold1_C1}, Threshold2={threshold2_C1}")
    print(f"Optimized Thresholds for C3: Threshold1={threshold1_C3}, Threshold2={threshold2_C3}")
    print(f"Optimized Thresholds for C4: Threshold1={threshold1_C4}, Threshold2={threshold2_C4}")

    # Generate fitted values using the stretched exponential model
    C1_fit = C1_func(strain, *popt_C1)
    C2_fit = C2_func(strain, *popt_C2)
    C3_fit = C3_func(strain, *popt_C3)
    C4_fit = C4_func(strain, *popt_C4)
        
    fitted_df = pd.DataFrame({
        'strain': strain,
        'C1': C1_fit,
        'C2': C2_fit,
        'C3': C3_fit,
        'C4': C4_fit
    })
    fitted_df.to_csv(f"Fitted_Parameters_{temp_file}.csv", index=False)

    # Calculate R^2 scores
    C_values = {'C1': C1_values, 'C2': C2_values, 'C3': C3_values, 'C4': C4_values}
    C_fits = {'C1': C1_fit, 'C2': C2_fit, 'C3': C3_fit, 'C4': C4_fit}
    r2_scores = {}

    for key in C_values:
        residuals = C_values[key] - C_fits[key]
        ss_res = np.sum(residuals**2)
        ss_tot = np.sum((C_values[key] - np.mean(C_fits[key]))**2)
        r2_scores[key] = 1 - (ss_res / ss_tot)

    
    # Plot C1 vs Strain 
    plt.figure(figsize=(10, 5))
    plt.plot(strain, C1_values, 'o', label='C1 Data')
    plt.plot(strain, C1_fit, '-', label='C1 Fit')
    plt.xlabel('Strain (PEEQ)')
    plt.ylabel('C1')
    plt.legend()
    plt.title(f"C1 vs Strain {temp_file} (R^2: {r2_scores['C1']:.3f})")
    plt.show()

    # Plot C2 vs Strain 
    plt.figure(figsize=(10, 5))
    plt.plot(strain[1:], C2_values[1:], 'o', label='C2 Data')
    plt.plot(strain[1:], C2_fit[1:], '-', label='C2 Fit')
    plt.xlabel('Strain (PEEQ)')
    plt.ylabel('C2')
    plt.legend()
    plt.title(f"C2 vs Strain {temp_file} (R^2: {r2_scores['C2']:.3f})")
    plt.show()

    # Plot C3 vs Strain 
    plt.figure(figsize=(10, 5))
    plt.plot(strain, C3_values, 'o', label='C3 Data')
    plt.plot(strain, C3_fit, '-', label='C3 Fit')
    plt.xlabel('Strain (PEEQ)')
    plt.ylabel('C3')
    plt.legend()
    plt.title(f"C3 vs Strain {temp_file} (R^2: {r2_scores['C3']:.3f})")
    plt.show()
    
     # Plot C4 vs Strain
    plt.figure(figsize=(10, 5))
    plt.plot(strain, C4_values, 'o', label='C4 Data')
    plt.plot(strain, C4_fit, '-', label='C4 Fit')
    plt.xlabel('Strain (PEEQ)')
    plt.ylabel('C4')
    plt.legend()
    plt.title(f"C4 vs Strain Fit {temp_file} (R^2: {r2_scores['C4']:.3f})")
    plt.show()

    # Print the fitted parameters for reference
    print("Parameters for C1:", popt_C1)
    print("Parameters for C2:", popt_C2)
    print("Parameters for C3:", popt_C3)
    print("Parameters for C4:", popt_C4)


In [ ]:
temperature_files = ['Tmp298.15_RD', 'Tmp373.15_RD', 'Tmp473.15_RD', 'Tmp573.15_RD', 'Tmp673.15_RD']

# Loop through each temperature file and plot C1 and C2 against strain (PEEQ)
for temp_file in temperature_files:
    calibrate_strain_dependent_dynamic(temp_file)

## Calibrate with fixed threshold

In [ ]:
# Define the stretched exponential model
def C1_func(x, a1, b1, c1, d1, e1, a2, b2, c2, a3, b3, c3,d3, e3,f3,g3, threshold1=0.002, threshold2=0.04):
    # Low strain: Oscillatory component
    small_strain = a1 * np.exp(b1 * x) + c1 + d1*x**2 + e1*x 
    
    # Middle strain: Oscillatory component for waves
    middle_strain = a3 * np.sin(b3 * x + c3) * np.exp(-d3 * x) + e3 * x**2 + f3 * x + g3 
    
    # High strain: Stable behavior
    high_strain = a2 * np.exp(b2 * x) + c2
    
    # Piecewise combination based on thresholds
    return np.where(
        x <= threshold1, small_strain,
        np.where(x <= threshold2, middle_strain, high_strain)
    )


def C2_func(x, a1, b1, c1, d1, a2, b2, c2, d2, a3, b3, c3, d3, threshold1=0.002, threshold2=0.04):
    epsilon = 1e-6  # To avoid log(0)
    x_shifted = x + epsilon

    # Small strain behavior
    small_strain = (
        a1 * np.log(x_shifted) + 
        b1 * x_shifted**c1 + 
        d1 * np.exp(-x_shifted)
    )
    
    # Middle strain behavior
    middle_strain = (
        a2 * np.log(x_shifted) + 
        b2 * x_shifted**c2 + 
        d2 * np.exp(-x_shifted)
    )
    
    # High strain behavior
    high_strain = (
        a3 * np.log(x_shifted) + 
        b3 * x_shifted**c3 + 
        d3 * np.exp(-x_shifted)
    )
    
    # Combine regions based on thresholds
    return np.where(
        x <= threshold1, small_strain,
        np.where(x <= threshold2, middle_strain, high_strain)
    )


def C3_func(x, a1, b1, c1, d1, a2, b2, c2, d2, a3,b3,c3, threshold1=0.002, threshold2=0.04):
        small_strain = a1*x**3 + b1*x**2 + c1*x + d1
        middle_strain = a2*x**3 + b2*x**2 + c2*x + d2
        high_strain = a3*x**2 + b3*x + c3
        # Piecewise combination based on thresholds
        return np.where(
            x <= threshold1, small_strain,
            np.where(x <= threshold2, middle_strain, high_strain)
        )


def C4_func(x, a1, b1, c1, d1, a2, b2, c2, a3, b3, c3,d3, e3,f3,g3, threshold1=0.002, threshold2=0.04):
        # Low strain: Oscillatory component
        small_strain = a1 * np.exp(b1 * x) * np.cos(c1 * x) + d1
        
        # Middle strain: Oscillatory component for waves
        middle_strain = a3 * np.sin(b3 * x + c3) * np.exp(-d3 * x) + e3 * x**2 + f3 * x + g3 
        
        # High strain: Stable behavior
        high_strain = a2 * np.exp(b2 * x) + c2
        
        # Piecewise combination based on thresholds
        return np.where(
            x <= threshold1, small_strain,
            np.where(x <= threshold2, middle_strain, high_strain)
        )


def calibrate_strain_dependent(temp_file):
    data = pd.read_csv(f"Parameters_{temp_file}_new.csv")  # Replace with your actual file path
    #data = data[data['PEEQ'] > 0.0009]
    strain = data['PEEQ']  # Strain values
    C1_values = data['C1']  # C1 values
    C2_values = data['C2']  # C2 values
    C3_values = data['C3']  # C1 values
    C4_values = data['C4']  # C2 values

    initial_guess_C1 = [0.1, -100, 100, 0.1, 1, 0.01, -0.5, -0.2, 0.02, 10, 0, 10, 0.01, -0.1, 0, 0.0015, 0.038]
    initial_guess_C2 = [1] * 12
    initial_guess_C3 = [1,1,1,1,1,1,1,1,1,1,1]
    initial_guess_C4 = [-0.1, -1000, 1000, 0, -0.01, -50, 0.01, 0.1, 1.0, 0.0, 0.1, 0.0, 1.0, 0.0]
    # Fit the stretched exponential model to C1
    popt_C1, _ = curve_fit(C1_func, strain, C1_values, p0=initial_guess_C1, maxfev=10000000)
    popt_C2, _ = curve_fit(C2_func, strain, C2_values, p0=initial_guess_C2, maxfev=10000000)
    popt_C3, _ = curve_fit(C3_func, strain, C3_values, p0=initial_guess_C3,maxfev=10000000)
    popt_C4, _ = curve_fit(C4_func, strain, C4_values, p0=initial_guess_C4, maxfev=10000000)


    # Generate fitted values using the stretched exponential model
    C1_fit = C1_func(strain, *popt_C1)
    C2_fit = C2_func(strain, *popt_C2)
    C3_fit = C3_func(strain, *popt_C3)
    C4_fit = C4_func(strain, *popt_C4)
        
    fitted_df = pd.DataFrame({
        'strain': strain,
        'C1': C1_fit,
        'C2': C2_fit,
        'C3': C3_fit,
        'C4': C4_fit
    })
    fitted_df.to_csv(f"Fitted_Parameters_{temp_file}.csv", index=False)

    # Calculate R^2 scores
    C_values = {'C1': C1_values, 'C2': C2_values, 'C3': C3_values, 'C4': C4_values}
    C_fits = {'C1': C1_fit, 'C2': C2_fit, 'C3': C3_fit, 'C4': C4_fit}
    r2_scores = {}

    for key in C_values:
        residuals = C_values[key] - C_fits[key]
        ss_res = np.sum(residuals**2)
        ss_tot = np.sum((C_values[key] - np.mean(C_fits[key]))**2)
        r2_scores[key] = 1 - (ss_res / ss_tot)

    # Plot C1 vs Strain 
    plt.figure(figsize=(7, 4))
    plt.plot(strain, C1_values, 'o', label='C1 Data', color='blue')
    plt.plot(strain, C1_fit, '-', label='C1 Fit', color='orange')
    plt.xlabel('Strain (PEEQ)')
    plt.ylabel('C1')
    plt.legend()
    plt.title(f"C1 vs Strain {temp_file} (R^2: {r2_scores['C1']:.3f})",fontweight='bold')
    plt.show()

    # Plot C2 vs Strain 
    plt.figure(figsize=(7, 4))
    plt.plot(strain[1:], C2_values[1:], 'o', label='C2 Data', color='blue')
    plt.plot(strain[1:], C2_fit[1:], '-', label='C2 Fit', color='orange')
    plt.xlabel('Strain (PEEQ)')
    plt.ylabel('C2')
    plt.legend()
    plt.title(f"C2 vs Strain {temp_file} (R^2: {r2_scores['C2']:.3f})", fontweight='bold')
    plt.show()

    # Plot C3 vs Strain 
    plt.figure(figsize=(7, 4))
    plt.plot(strain, C3_values, 'o', label='C3 Data', color='blue')
    plt.plot(strain, C3_fit, '-', label='C3 Fit', color='orange')
    plt.xlabel('Strain (PEEQ)')
    plt.ylabel('C3')
    plt.legend()
    plt.title(f"C3 vs Strain {temp_file} (R^2: {r2_scores['C3']:.3f})", fontweight='bold')
    plt.show()
    
     # Plot C4 vs Strain
    plt.figure(figsize=(7, 4))
    plt.plot(strain, C4_values, 'o', label='C4 Data', color='blue')
    plt.plot(strain, C4_fit, '-', label='C4 Fit', color='orange')
    plt.xlabel('Strain (PEEQ)')
    plt.ylabel('C4')
    plt.legend()
    plt.title(f"C4 vs Strain Fit {temp_file} (R^2: {r2_scores['C4']:.3f})", fontweight='bold')
    plt.show()

    # Print the fitted parameters for reference
    print("Parameters for C1:", popt_C1)
    print("Parameters for C2:", popt_C2)
    print("Parameters for C3:", popt_C3)
    print("Parameters for C4:", popt_C4)

In [ ]:
temperature_files = ['Tmp298.15_RD', 'Tmp373.15_RD', 'Tmp473.15_RD', 'Tmp573.15_RD', 'Tmp673.15_RD']

# Loop through each temperature file and plot C1 and C2 against strain (PEEQ)
for temp_file in temperature_files:
    calibrate_strain_dependent(temp_file)

# Validate function with calibrated parameters

In [ ]:
def validate_original_function(temperature_file, strain_rates, strain_rate_columns):
    """Validates the original function using the fitted parameters and calculates R-value."""
    import matplotlib.pyplot as plt
    from cycler import cycler

    # Load the original data and the fitted parameters
    data = pd.read_csv(f"../{temperature_file}.csv")
    params_df = pd.read_csv(f"Parameters_{temperature_file}_new.csv")

    # Filter the parameters DataFrame for specific strain values (from the picture)
    target_strains = [0.00011, 0.0002, 0.0003, 0.0004, 0.0005]
    #target_strains = [0.01, 0.02, 0.03, 0.04, 0.05]
    params_df = params_df[params_df['PEEQ'].isin(target_strains)]

    # Define color scheme (matching the example plot)
    colors = ["blue", "red", "green", "purple", "orange"]

    plt.figure(figsize=(8, 6))
    plt.gca().set_prop_cycle(cycler('color', colors))

    for color, (_, row) in zip(colors, params_df.iterrows()):
        # Extract the parameters and stress data for comparison
        strain = row['PEEQ']
        C1, C2, C3, C4 = row[['C1', 'C2', 'C3', 'C4']]
        stress_values = data.iloc[int(row.name)][strain_rate_columns].values

        # Generate smooth strain rate values for plotting
        plt_sr = np.linspace(min(strain_rates), max(strain_rates), 100)
        plt_stress = strain_rate_dependent_stress(plt_sr, C1, C2, C3, C4)

        # Plot for visual validation
        plt.plot(strain_rates, stress_values, 'o', label=f'Data', color=color)
        plt.plot(plt_sr, plt_stress, '-', label=f'Fit (Strain={strain:.5f})', color=color)

    # Style adjustments for the plot
    plt.xlabel('Strain-rate (1/s)', fontsize=12)
    plt.ylabel('Normalized Stress', fontsize=12)
    plt.ylim(0.75,2.2)
    plt.title(f'{temperature_file} low strain', fontsize=14)
    plt.legend(loc='best',fontsize=10)
    plt.show()

# Example usage
temperature_file = 'Tmp298.15_RD'
strain_rates = [0.0001, 0.001, 0.01, 0.1, 1]
strain_rate_columns = ['ref_StrRt0.0001', 'ref_StrRt0.001', 'ref_StrRt0.01', 'ref_StrRt0.1', 'ref_StrRt1']
validate_original_function(temperature_file, strain_rates, strain_rate_columns)

temps = ['373.15', '473.15', '573.15', '673.15']
strain_rates = [0.0001, 0.001, 0.01, 0.1]
strain_rate_columns = ['ref_StrRt0.0001', 'ref_StrRt0.001', 'ref_StrRt0.01', 'ref_StrRt0.1']
for temp in temps:
    validate_original_function(f"Tmp{temp}_RD", strain_rates, strain_rate_columns)


In [ ]:
def check_r_squared(temperature_files):
    """
    Checks R_squared values in the Parameters file for each temperature and reports the count
    and percentage of rows with R_squared below a specified threshold.
    
    Parameters:
    - temperature_files (list of str): List of temperature file names (without ".csv" extension).
    
    Output:
    - Prints the count and percentage of rows with R_squared < threshold for each temperature file.
    """
    threshold = 0.80
    for temp_file in temperature_files:
        # Load the data from the CSV file
        df = pd.read_csv(f"R_values_{temp_file}.csv")
       
        plt.plot(df['Row'],df['R_squared'])
        plt.title(f"Validation R squared for {temp_file}", fontweight='bold')
        plt.xlabel('PEEQ')
        plt.ylabel('R squared')
        plt.ylim(0,2)
        plt.show()
        # Count rows where R_squared is less than the threshold
        total_rows = len(df)
        count_low_r_squared = df[df['R_squared'] > threshold].shape[0]
        
        # Calculate the percentage of rows with low R_squared
        percentage_low_r_squared = (count_low_r_squared / total_rows) * 100 if total_rows > 0 else 0
        
        # Print results
        print(f"\nTemperature File: Parameters_{temp_file}.csv")
        if count_low_r_squared > 0:
            print(f"Number of rows with R_squared < {threshold}: {count_low_r_squared}")
            print(f"Percentage of rows with R_squared < {threshold}: {percentage_low_r_squared:.2f}%")
        else:
            print(f"All R_squared values are >= {threshold}.")

        r = max(df['R_squared'])
        r_m = min(df['R_squared'])
        r_a = sum(df['R_squared'])/total_rows
        print(f'Max = {r}. Average = {r_m}')

# Usage
temperature_files = ['Tmp298.15_RD', 'Tmp373.15_RD', 'Tmp473.15_RD', 'Tmp573.15_RD', 'Tmp673.15_RD']  # List of temperature files to check
check_r_squared(temperature_files)


## Extrapolate: Extend C1-C4 for strain 0-3

In [ ]:
# 473.15K
C1 = [
    -8.23735520e-01, -3.21124150e+04, 3.50201203e-01, 1.12558008e+06,
    -1.61848397e+03, 7.02124262e-01, -2.00924548e-01, -9.04805111e-01,
    1.22263074e-01, -2.66849185e+01, -1.18313396e+00, -5.92075346e+01,
    4.47497201e+02, 4.48368357e+00, -8.43978925e-02, 1.50000000e-03,
    3.80000000e-02
]

C2 = [
    1.47699645e-02, -4.86876323e+00, 5.55396313e-01, 1.56151734e+00,
    -3.17818220e-02, 1.38843372e+04, 4.01423154e+00, 1.16371818e+00,
    -3.18779876e-01, 3.16619726e+00, 3.71200374e-01, -7.59985603e-01
]

C3 = [
    6.33581498e+08, -2.17011734e+06, 2.13477619e+03, -3.37956146e-01,
    4.55094602e+03, -3.09265375e+02, 5.37886321e+00, 3.44301977e-01,
    2.99549059e+00, 3.88154615e-01, 3.30577369e-01
]

C4 = [
    -2.30800768e-01, -1.04967791e+04, -7.73432274e+03, -9.38066754e-03,
    -4.07611967e-04, 1.18586353e+01, -1.03573295e-02, 1.19837316e+00,
    -1.83765841e+01, 1.31131752e+00, -4.58470496e+00, 2.32431653e+02,
    3.18446069e-02, -1.16855798e+00
]


# Load strain rates and stress values for extrapolation
strain_rates = [0.0001, 0.001, 0.01, 0.1, 1]  # Example strain rates
strain = pd.read_csv('../../extended_curve/Tmp233.15K_StrRt0.0001_RD.csv')['Strain']

# Generate parameter values using provided functions
params_df = pd.DataFrame({
    'strain': strain,
    'C1': C1_func(strain, *C1),
    'C2': C2_func(strain, *C2),
    'C3': C3_func(strain, *C3),
    'C4': C4_func(strain, *C4)
})

# Prepare for saving stress values
stress_results = []

# Iterate through each row to compute and save stress for different strain rates
for i in range(len(params_df)):
    # Extract strain and parameter values
    current_strain = params_df.iloc[i]['strain']
    C1_val, C2_val, C3_val, C4_val = params_df.iloc[i][['C1', 'C2', 'C3', 'C4']]
    
    # Compute stress for each strain rate
    stress_values = strain_rate_dependent_stress(np.array(strain_rates), C1_val, C2_val, C3_val, C4_val)
    
    # Store results with corresponding strain
    stress_results.append({
        'strain': current_strain,
        **{f'stress_strRt_{sr}': stress for sr, stress in zip(strain_rates, stress_values)}
    })

# Save stress results to a DataFrame and export to CSV
stress_df = pd.DataFrame(stress_results)
output_filename = 'Stress_Extrapolated_473.15K.csv'
stress_df.to_csv(output_filename, index=False)

print(f"Extrapolated stress values saved to {output_filename}.")


In [ ]:
# Load the files
file_473_path = 'Stress_Extrapolated_473.15K.csv'
file_298_path = '../../extended_curve/Tmp298.15K_StrRt0.0001_RD.csv'

# Read the CSV files into pandas DataFrames
data_473 = pd.read_csv(file_473_path)
data_298 = pd.read_csv(file_298_path)

# Ensure consistent column naming for merging
data_473.rename(columns={"strain": "Strain"}, inplace=True)

# Merge the datasets on 'Strain'
merged_data = pd.merge(data_473, data_298, on="Strain", how="inner")

# Calculate the product of stress values
merged_data["Stress_Product"] = merged_data["stress_strRt_0.0001"] * merged_data["Stress"]

# Save the result to a new CSV file (optional)
merged_data.to_csv('Stress_Product_Output.csv', index=False)

In [ ]:
def validate_original_function(temperature_file, strain_rates, strain_rate_columns):
    """Validates the original function using the fitted parameters and calculates R-value."""
    import matplotlib.pyplot as plt
    from cycler import cycler

    # Load the original data and the fitted parameters
    data = pd.read_csv(f"../{temperature_file}.csv")
    params_df = pd.read_csv(f"Fitted_Parameters_{temperature_file}.csv")

    # Filter the parameters DataFrame for specific strain values (from the picture)
    target_strains = [0.00011, 0.0002, 0.0003, 0.0004, 0.0005]
    target_strains = [0.01, 0.02, 0.03, 0.04, 0.05]

    params_df = params_df[params_df['strain'].isin(target_strains)]

    # Define color scheme (matching the example plot)
    colors = ["blue", "red", "green", "purple", "orange"]

    plt.figure(figsize=(8, 6))
    plt.gca().set_prop_cycle(cycler('color', colors))

    for color, (_, row) in zip(colors, params_df.iterrows()):
        # Extract the parameters and stress data for comparison
        strain = row['strain']
        C1, C2, C3, C4 = row[['C1', 'C2', 'C3', 'C4']]
        stress_values = data.iloc[int(row.name)][strain_rate_columns].values

        # Generate smooth strain rate values for plotting
        plt_sr = np.linspace(min(strain_rates), max(strain_rates), 100)
        plt_stress = strain_rate_dependent_stress(plt_sr, C1, C2, C3, C4)

        # Plot for visual validation
        plt.plot(strain_rates, stress_values, 'o', label=f'Data', color=color)
        plt.plot(plt_sr, plt_stress, '-', label=f'Fit (Strain={strain:.5f})', color=color)

    # Style adjustments for the plot
    plt.xlabel('Strain-rate (1/s)', fontsize=12)
    plt.ylabel('Normalized Stress', fontsize=12)
    plt.title(f'{temperature_file} high strain', fontsize=14)
    plt.legend(loc='best',fontsize=10)
    plt.show()

# Example usage
temperature_file = 'Tmp298.15_RD'
strain_rates = [0.0001, 0.001, 0.01, 0.1, 1]
strain_rate_columns = ['ref_StrRt0.0001', 'ref_StrRt0.001', 'ref_StrRt0.01', 'ref_StrRt0.1', 'ref_StrRt1']
validate_original_function(temperature_file, strain_rates, strain_rate_columns)

temps = ['373.15', '473.15', '573.15', '673.15']
strain_rates = [0.0001, 0.001, 0.01, 0.1]
strain_rate_columns = ['ref_StrRt0.0001', 'ref_StrRt0.001', 'ref_StrRt0.01', 'ref_StrRt0.1']
for temp in temps:
    validate_original_function(f"Tmp{temp}_RD", strain_rates, strain_rate_columns)
